In [2]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
encoder = LabelEncoder()
encoder.classes_ = np.load('classes_two.npy',allow_pickle=True)
print(encoder.classes_ )

['non-toxic' 'toxic']


In [6]:
from collections import Counter,defaultdict
from tqdm.notebook import tqdm
import json
import pandas as pd

In [4]:
def get_annotated_data(params):
    #temp_read = pd.read_pickle(params['data_file'])
    with open(params['data_file'], 'r') as fp:
        data = json.load(fp)
    dict_data=[]
    for key in data:
        temp={}
        temp['post_id']=key
        temp['text']=data[key]['post_tokens']
        final_label=[]
        for i in range(1,4):
            temp['annotatorid'+str(i)]=data[key]['annotators'][i-1]['annotator_id']
#             temp['explain'+str(i)]=data[key]['annotators'][i-1]['rationales']
            temp['target'+str(i)]=data[key]['annotators'][i-1]['target']
            temp['label'+str(i)]=data[key]['annotators'][i-1]['label']
            final_label.append(temp['label'+str(i)])

        final_label_id=max(final_label,key=final_label.count)
        temp['rationales']=data[key]['rationales']
            
        if(params['class_names']=='Data/classes_two.npy'):
            if(final_label.count(final_label_id)==1):
                temp['final_label']='undecided'
            else:
                if(final_label_id in ['hatespeech','offensive']):
                    final_label_id='toxic'
                else:
                    final_label_id='non-toxic'
                temp['final_label']=final_label_id

        
        else:
            if(final_label.count(final_label_id)==1):
                temp['final_label']='undecided'
            else:
                temp['final_label']=final_label_id

        
        
        
        dict_data.append(temp)    
    temp_read = pd.DataFrame(dict_data)  
    return temp_read    

In [7]:
dict_data_folder={
      '2':{'data_file':'dataset.json','class_label':'classes_two.npy'},
      #'3':{'data_file':'dataset.json','class_label':'classes.npy'}
}

params = {}

# We need to load the dataset with the labels as 'toxic' and 'non-toxic'. 
# We consider hatespeech and offensive as toxic and normal as non-toxic.
params['num_classes']=2  
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [8]:
data_all_labelled

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",4,[African],normal,2,[None],normal,3,[African],hatespeech,[],normal
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",1,[Asian],hatespeech,4,[Asian],offensive,3,[Asian],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
4,1179085312976445440_twitter,"[this, bitch, in, whataburger, eating, a, burg...",4,"[Caucasian, Women]",hatespeech,2,"[Women, Caucasian]",hatespeech,3,"[Women, Caucasian]",offensive,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"[if, ur, still, on, twitter, tell, carlton, i,...",217,"[Men, Women, Other]",offensive,199,[None],offensive,215,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",offensive
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",220,[African],offensive,223,"[African, Other]",offensive,231,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",offensive
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",206,[Islam],offensive,203,[Other],normal,211,[None],normal,[],normal
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",209,[Hispanic],hatespeech,253,[Asian],hatespeech,222,[Asian],offensive,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",hatespeech


In [15]:
def generate_target_information(dataset):
    final_target_output = defaultdict(list)
    all_communities_selected = []
    
    for each in dataset.iterrows(): 
        # All the target communities tagged for this post
        all_targets = each[1]['target1']+each[1]['target2']+each[1]['target3']  
        community_dict = dict(Counter(all_targets))
        
        # Select only those communities which are present more than once.
        for key in community_dict:
            if community_dict[key]>1:  
                final_target_output[each[1]['post_id']].append(key)
                all_communities_selected.append(key)
        
        # If no community is selected based on majority voting then we don't select any community
        if each[1]['post_id'] not in final_target_output:
            final_target_output[each[1]['post_id']].append('None')
            all_communities_selected.append(key)

    return final_target_output, all_communities_selected

In [16]:
target_information, all_communities_selected = generate_target_information(data_all_labelled)

In [18]:
all_communities_selected

['None',
 'None',
 'African',
 'Asian',
 'Caucasian',
 'Women',
 'Jewish',
 'African',
 'African',
 'Jewish',
 'Homosexual',
 'African',
 'Jewish',
 'African',
 'Islam',
 'None',
 'None',
 'None',
 'Women',
 'None',
 'None',
 'None',
 'None',
 'Jewish',
 'None',
 'None',
 'None',
 'Jewish',
 'African',
 'None',
 'Caucasian',
 'Hispanic',
 'Jewish',
 'Hispanic',
 'None',
 'African',
 'Islam',
 'Homosexual',
 'None',
 'None',
 'None',
 'Women',
 'African',
 'Women',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'Homosexual',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'Homosexual',
 'None',
 'African',
 'Homosexual',
 'Hispanic',
 'Islam',
 'Homosexual',
 'Refugee',
 'African',
 'African',
 'Jewish',
 'Disability',
 'African',
 'Jewish',
 'African',
 'Islam',
 'Jewish',
 'African',
 'Jewish',
 'African',
 'Arab',
 'Islam',
 'Homosexual',
 'None',
 'Islam',
 'Jewish',
 'Hispanic',
 'Jewish',
 'None',
 'None',
 'Women',
 'Non

In [17]:
target_information

defaultdict(list,
            {'1179055004553900032_twitter': ['None'],
             '1179063826874032128_twitter': ['None'],
             '1178793830532956161_twitter': ['African'],
             '1179088797964763136_twitter': ['Asian'],
             '1179085312976445440_twitter': ['Caucasian', 'Women'],
             '13851720_gab': ['Jewish'],
             '24198545_gab': ['African'],
             '14567516_gab': ['African', 'Jewish', 'Homosexual'],
             '19679648_gab': ['African', 'Jewish'],
             '15858421_gab': ['African'],
             '23844035_gab': ['Islam'],
             '1089312238102609921_twitter': ['None'],
             '1159278048069464065_twitter': ['None'],
             '1170440965888335874_twitter': ['None'],
             '1104904416430194688_twitter': ['Women'],
             '1122734457230774272_twitter': ['None'],
             '1084835380619300864_twitter': ['None'],
             '1178818409812746240_twitter': ['None'],
             '117890533838656716

In [21]:
community_count_dict = Counter(all_communities_selected)

# We remove None and Other from dictionary
community_count_dict.pop('None')
community_count_dict.pop('Other')

# For the bias calculation, we are considering the top 10 communites based on their count
list_selected_community = [community for community, value in community_count_dict.most_common(10)]
list_selected_community

['African',
 'Islam',
 'Jewish',
 'Homosexual',
 'Women',
 'Refugee',
 'Arab',
 'Caucasian',
 'Asian',
 'Hispanic']

In [22]:
final_target_information ={}
for each in target_information:
    temp = list(set(target_information[each])&set(list_selected_community))
    if len(temp) == 0:
        final_target_information[each] = None
    else:
        final_target_information[each] = temp

In [23]:
# Add a new column 'final_target_category' which will contain the selected target community names
data_all_labelled['final_target_category'] = data_all_labelled['post_id'].map(final_target_information)

In [32]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
postpost_id_divisions_path = 'post_id_divisions.json'

with open(postpost_id_divisions_path, 'r') as fp:
    post_id_dict=json.load(fp)

data_all_labelled_bias_test = data_all_labelled[data_all_labelled['post_id'].isin(post_id_dict['test'])]

data_all_labelled_bias_test.to_pickle("../../test.pkl")
data_all_labelled_bias_test

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label,final_target_category
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal,None
5,13851720_gab,"[laura, loomer, raped, me, while, screaming, a...",1,[Jewish],hatespeech,2,[Jewish],hatespeech,3,[Jewish],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,...",hatespeech,[Jewish]
17,1178818409812746240_twitter,"[<user>, what, did, the, old, lady, do, was, s...",9,[None],normal,10,[None],normal,4,[None],normal,[],normal,None
25,19346774_gab,"[as, much, as, i, appreciate, a, format, of, o...",9,[None],normal,13,[None],normal,4,[Hispanic],offensive,[],normal,None
36,1165819983701643266_twitter,"[sex, be, so, good, a, bitch, be, slow, stroki...",4,[Women],offensive,7,[Women],offensive,16,[None],normal,"[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, ...",offensive,[Women]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20105,9773208_gab,"[it, always, women, trying, this, shit, like, ...",200,[Women],hatespeech,202,[Women],offensive,203,[Women],offensive,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",offensive,[Women]
20109,9802943_gab,"[because, women, would, never, lie, about, bei...",228,[Women],offensive,222,[Women],offensive,209,[Women],normal,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",offensive,[Women]
20117,9826963_gab,"[how, is, all, that, awesome, muslim, diversit...",228,[Islam],offensive,222,[Islam],offensive,209,[Islam],offensive,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",offensive,[Islam]
20128,9867117_gab,"[well, my, dear, lgbtq, brothers, and, sisters...",207,"[Islam, Other]",offensive,223,[Islam],hatespeech,231,[Islam],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech,[Islam]


In [33]:
data_all_labelled_bias_train = data_all_labelled[data_all_labelled['post_id'].isin(post_id_dict['train'])]

data_all_labelled_bias_train.to_pickle("../../training.pkl")
data_all_labelled_bias_train

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label,final_target_category
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal,None
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",4,[African],normal,2,[None],normal,3,[African],hatespeech,[],normal,[African]
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",1,[Asian],hatespeech,4,[Asian],offensive,3,[Asian],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech,[Asian]
6,24198545_gab,"[and, this, is, why, i, end, up, with, nigger,...",4,[African],hatespeech,3,[African],hatespeech,5,[African],offensive,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,...",hatespeech,[African]
7,14567516_gab,"[nogs, jews, and, dykes, how, enriching]",4,"[African, Jewish, Homosexual]",offensive,3,"[African, Jewish, Homosexual]",hatespeech,5,[Jewish],offensive,"[[1, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0], [1, 1...",offensive,"[Homosexual, Jewish, African]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20142,9988840_gab,"[this, vile, racist, ghetto, ass, dumber, than...",199,[None],offensive,222,[Women],offensive,209,[Women],offensive,"[[0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,...",offensive,[Women]
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",220,[African],offensive,223,"[African, Other]",offensive,231,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",offensive,[African]
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",206,[Islam],offensive,203,[Other],normal,211,[None],normal,[],normal,None
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",209,[Hispanic],hatespeech,253,[Asian],hatespeech,222,[Asian],offensive,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",hatespeech,[Asian]
